In [1]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow_model_optimization as tfmot
import tempfile
import tensorflow as tf
from keras.layers import Input, Dense, BatchNormalization, Reshape, Conv2D, add, LeakyReLU
from keras.models import Model, model_from_json
from keras.callbacks import TensorBoard, Callback
import scipy.io as sio 
import numpy as np
import math
import time
tf.compat.v1.reset_default_graph()

In [2]:
envir = 'indoor' #'indoor' or 'outdoor'
# image params
img_height = 32
img_width = 32
img_channels = 2 
img_total = img_height*img_width*img_channels
# network params
residual_num = 2
encoded_dim = 512 #compress rate=1/4->dim.=512, compress rate=1/16->dim.=128, compress rate=1/32->dim.=64, compress rate=1/64->dim.=32

In [3]:
# Data loading
if envir == 'indoor':
    mat = sio.loadmat('data/DATA_Htrainin.mat') 
    x_train = mat['HT'] # array
    mat = sio.loadmat('data/DATA_Hvalin.mat')
    x_val = mat['HT'] # array
    mat = sio.loadmat('data/DATA_Htestin.mat')
    x_test = mat['HT'] # array

elif envir == 'outdoor':
    mat = sio.loadmat('data/DATA_Htrainout.mat') 
    x_train = mat['HT'] # array
    mat = sio.loadmat('data/DATA_Hvalout.mat')
    x_val = mat['HT'] # array
    mat = sio.loadmat('data/DATA_Htestout.mat')
    x_test = mat['HT'] # array

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
x_train = np.reshape(x_train, (len(x_train), img_channels, img_height, img_width))  # adapt this if using `channels_first` image data format
x_val = np.reshape(x_val, (len(x_val), img_channels, img_height, img_width))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), img_channels, img_height, img_width))  # adapt this if using `channels_first` image data format

In [4]:
file = 'CsiNet_'+(envir)+'_dim'+str(encoded_dim)
json_file = open('saved_model/model_%s.json'%file, 'r')
base_model_json = json_file.read()
json_file.close()
base_model = model_from_json(base_model_json)
# load weights into new model
base_model.load_weights("saved_model/model_%s.h5"%file)
print("Loaded base model from disk")
base_model.summary()    

Loaded base model from disk
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2, 32, 32)]  0           []                               
                                                                                                  
 conv2d_1 (Conv2D)              (None, 2, 32, 32)    38          ['input_1[0][0]']                
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 2, 32, 32)   128         ['conv2d_1[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 leaky_re_lu_1 (LeakyReLU)      (None, 2, 32, 32)    0          

In [5]:
# def clone_fn(layer):
#     if layer.name == 'conv2d_2' or 'conv2d_3' or 'conv2d_4':
#         # The default padding `SAME` for the first convolution is incompatible
#         # with XNNPACK sparse inference.
#         layer.padding = 'valid'
#         layer.kernel_size=(1,1)
#         # We ask the model to rebuild since we've changed the padding parameter.
#         layer.built = False
#     return layer

# a = tf.keras.models.clone_model(base_model, clone_function=clone_fn)
# a.summary()

In [6]:
# Helper function uses `prune_low_magnitude` to make only the 
# Dense layers train with pruning.
def apply_pruning_to_dense(layer):
  pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(0.1, begin_step=0, frequency=100), #Control sparsity
      # 'pruning_policy': tfmot.sparsity.keras.PruneForLatencyOnXNNPack()
  }
  if isinstance(layer, tf.keras.layers.Dense):
    return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
  return layer

# Use `tf.keras.models.clone_model` to apply `apply_pruning_to_dense` 
# to the layers of the model.
model_for_pruning = tf.keras.models.clone_model(
    base_model,
    clone_function=apply_pruning_to_dense,
)

model_for_pruning.summary()
# # Define parameters for pruning.
# pruning_params = {
#       'pruning_policy': tfmot.sparsity.keras.PruneForLatencyOnXNNPack()
# }
# prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
# # Try to apply pruning wrapper with pruning policy parameter.
# model_for_pruning = prune_low_magnitude(base_model, **pruning_params)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2, 32, 32)]  0           []                               
                                                                                                  
 conv2d_1 (Conv2D)              (None, 2, 32, 32)    38          ['input_1[0][0]']                
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 2, 32, 32)   128         ['conv2d_1[1][0]']               
 rmalization)                                                                                     
                                                                                                  
 leaky_re_lu_1 (LeakyReLU)      (None, 2, 32, 32)    0           ['batch_normalization_1[1][

In [7]:
# Compiling model for pruning.
model_for_pruning.compile(optimizer='adam', loss='mse')
log_dir = tempfile.mkdtemp()
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    # Log sparsity and other metrics in Tensorboard.
    tfmot.sparsity.keras.PruningSummaries(log_dir=log_dir)
]

In [8]:
model_for_pruning.fit(x_train, x_train,
                epochs=15,
                batch_size=200,
                shuffle=True,
                validation_data=(x_val, x_val),
                callbacks= callbacks)

Epoch 1/15
500/500 [==============================] - 28s 40ms/step - loss: 9.2697e-06 - val_loss: 8.1164e-06
Epoch 2/15
500/500 [==============================] - 18s 37ms/step - loss: 7.5190e-06 - val_loss: 8.4994e-06
Epoch 3/15
500/500 [==============================] - 18s 37ms/step - loss: 7.5571e-06 - val_loss: 7.7751e-06
Epoch 4/15
500/500 [==============================] - 18s 37ms/step - loss: 7.4059e-06 - val_loss: 7.7134e-06
Epoch 5/15
500/500 [==============================] - 19s 37ms/step - loss: 7.4055e-06 - val_loss: 7.6242e-06
Epoch 6/15
500/500 [==============================] - 18s 37ms/step - loss: 7.3249e-06 - val_loss: 7.5572e-06
Epoch 7/15
500/500 [==============================] - 19s 37ms/step - loss: 7.3411e-06 - val_loss: 7.7668e-06
Epoch 8/15
500/500 [==============================] - 19s 38ms/step - loss: 7.2710e-06 - val_loss: 7.3989e-06
Epoch 9/15
500/500 [==============================] - 19s 39ms/step - loss: 7.3210e-06 - val_loss: 7.7685e-06
Epoch 10/1

In [9]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)


In [10]:
file = (envir)+'_dim'+str(encoded_dim)
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
# pruned_keras_file = 'optimized_models/%s/pruned_model_%s.h5'%(file, file)
# tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
# print('Saved pruned Keras model to:', pruned_keras_file)
def print_model_weights_sparsity(model):

    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Wrapper):
            weights = layer.trainable_weights
        else:
            weights = layer.weights
        for weight in weights:
            # ignore auxiliary quantization weights
            if "quantize_layer" in weight.name:
                continue
            weight_size = weight.numpy().size
            zero_num = np.count_nonzero(weight == 0)
            print(
                f"{weight.name}: {zero_num/weight_size:.2%} sparsity ",
                f"({zero_num}/{weight_size})",
            )
print_model_weights_sparsity(model_for_export)

conv2d_1/kernel:0: 0.00% sparsity  (0/36)
conv2d_1/bias:0: 0.00% sparsity  (0/2)
batch_normalization_1/gamma:0: 0.00% sparsity  (0/32)
batch_normalization_1/beta:0: 0.00% sparsity  (0/32)
batch_normalization_1/moving_mean:0: 0.00% sparsity  (0/32)
batch_normalization_1/moving_variance:0: 0.00% sparsity  (0/32)
dense_1/kernel:0: 30.00% sparsity  (314573/1048576)
dense_1/bias:0: 0.00% sparsity  (0/512)
dense_2/kernel:0: 30.00% sparsity  (314573/1048576)
dense_2/bias:0: 0.00% sparsity  (0/2048)
conv2d_2/kernel:0: 0.00% sparsity  (0/144)
conv2d_2/bias:0: 0.00% sparsity  (0/8)
batch_normalization_2/gamma:0: 0.00% sparsity  (0/32)
batch_normalization_2/beta:0: 0.00% sparsity  (0/32)
batch_normalization_2/moving_mean:0: 0.00% sparsity  (0/32)
batch_normalization_2/moving_variance:0: 0.00% sparsity  (0/32)
conv2d_3/kernel:0: 0.00% sparsity  (0/1152)
conv2d_3/bias:0: 0.00% sparsity  (0/16)
batch_normalization_3/gamma:0: 0.00% sparsity  (0/32)
batch_normalization_3/beta:0: 0.00% sparsity  (0/32)

In [16]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.EXPERIMENTAL_SPARSITY]
pruned_tflite_model = converter.convert()

# Saving the model.
with open('optimized_models/%s/pruned_30_model_%s.tflite'%(file, file), 'wb') as f:
  f.write(pruned_tflite_model)


INFO:tensorflow:Assets written to: C:\Users\Omar\AppData\Local\Temp\tmpy8b46qxa\assets


INFO:tensorflow:Assets written to: C:\Users\Omar\AppData\Local\Temp\tmpy8b46qxa\assets


In [17]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [18]:
prune_size = (get_gzipped_model_size('optimized_models/%s/pruned_30_model_%s.tflite'%(file, file)))
print("pruned model in Mb:", prune_size / float(2**20))

pruned model in Mb: 5.755962371826172


In [19]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

# Passing the Keras model to the TF Lite Converter.
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
# Using float-16 quantization.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
# Converting the model.
pruned_fp16_quantized_tflite_model = converter.convert()

# Saving the model.
with open('optimized_models/%s/pruned_30_quant_model_%s.tflite'%(file, file), 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)
with open('optimized_models/%s/pruned_30_fp16_quant_model_%s.tflite'%(file, file), 'wb') as f:
  f.write(pruned_fp16_quantized_tflite_model)



INFO:tensorflow:Assets written to: C:\Users\Omar\AppData\Local\Temp\tmpp1u3b2z1\assets


INFO:tensorflow:Assets written to: C:\Users\Omar\AppData\Local\Temp\tmpp1u3b2z1\assets


INFO:tensorflow:Assets written to: C:\Users\Omar\AppData\Local\Temp\tmp_9xhjsz_\assets


INFO:tensorflow:Assets written to: C:\Users\Omar\AppData\Local\Temp\tmp_9xhjsz_\assets


In [21]:
prune_quant_size = (get_gzipped_model_size('optimized_models/%s/pruned_30_quant_model_%s.tflite'%(file, file)))
prune_fp16_quant_size = (get_gzipped_model_size('optimized_models/%s/pruned_30_fp16_quant_model_%s.tflite'%(file, file)))
print("pruned quant model in Mb:", prune_quant_size / float(2**20))

pruned quant model in Mb: 1.0977869033813477
